In [71]:
import numpy as np
import copy
import pickle
import math
from pennylane.labs import resource_estimation as re
from pennylane.labs.trotter_error import vibronic_fragments
from pennylane.labs.trotter_error.product_formulas import trotter_error
from pennylane.labs.trotter_error import RealspaceMatrix

# Quantum algorithm for vibronic dynamics

### Vibronic dynamics

While ground state chemistry can often be described within the adiabatic Born-Oppenheimer (BO) approximation, where nuclei evolve on a single electronic potential energy surface (PES) that is energetically distant from others, **excited-state processes** encountered in photochemistry often require a description involving nuclear dynamics on multiple potential energy surfaces. This is due to the frequently encountered lack of energetic separation within the excited state manifold, and that non-adiabatic couplings can become substantial in the viscinity of degeneracy. In order to simulate the non-adiabatic dynamics relevant to various photodynamical applications, it is common to employ a vibronic coupling Hamiltonian, which describes the interaction between a set of electronic states and nuclear vibrational motion. A **vibronic Hamiltonian** comprising $N$ electronic states and $M$ vibrational modes is expressed as

$$
H = T + V,
$$

where, in the diabatic electronic basis, kinetic energy $T$ is diagonal while potential $V$ contains off-diagonal electronic couplings,

$$
T = \mathbb{I}_{\text{el}} \otimes \sum_{r=0}^{M-1} \frac{\omega_r}{2} P_r^2, \quad
V = (\mathbb{I}_{\text{el}} \otimes V_0) + W' = \sum_{i,j=0}^{N-1} |j\rangle \langle i| \otimes V_{ji},
$$

where $\mathbb{I}_{\text{el}}$ is the $N \times N$ identity matrix acting on the electronic space, $\omega_r$ is the harmonic frequency of the $r^{\rm{th}}$ vibrational normal mode, with associated momentum and position operators $P_r$ and $Q_r$ respectively. Couplings $V_{ji}$ are generally given by degree $d$ multivariate polynomials of position operators,

$$
V_{ji} = \sum_{|\vec{\alpha}| \leq d} c_{\vec{\alpha}}^{(j,i)} \mathbf{Q}^{\vec{\alpha}} = \lambda^{(j,i)} + \sum_{r=0}^{M-1} \gamma_r^{(j,i)} Q_r + \sum_{r,r'=0}^{M-1} \beta_{rr'}^{(j,i)} Q_r Q_{r'} + \cdots
$$

where we have used the multi-index notation:

$$
\mathbf{Q}^{\vec{\alpha}} = Q_0^{\alpha_0} Q_1^{\alpha_1} \cdots Q_{M-1}^{\alpha_{M-1}}.
$$

Typical truncations are $d=1$ and $d=2$, which result in the commonly employed linear vibronic coupling (LVC) and quadratic vibronic coupling models (QVC), respectively. While LVCs are already sufficient to model qualitative features of the vibronic interaction, such as the existence of a conical intersection between PESs, higher order expansions are required for the quantitative determination of observables, particularly over dynamics involving longer timescales, large amplitude nuclear motions, or highly anharmonic electronic potentials.

### Observable quantities

Performing time evolution of an initial vibronic wavefunction $| \psi(0) \rangle$ enables the calculation of various observables, such as vibronically resolved spectra and electronic state populations as a function of time. In the context of molecular photodynamics, the latter is especially important, as it allows for the explicit, non-perturbative description of quantum dynamical processes such as exciton and charge transfer, internal conversions, and intersystem crossings. The population of the wavefunction $| \psi (t) \rangle = e^{-i H t} | \psi (0) \rangle$ in the $i^{\rm{th}}$ electronic state is simply given by

$$
p_i(t) = \langle \psi (t) | \left( | i \rangle \langle i | \otimes \mathbb{I}_{vib} \right) | \psi (t) \rangle,
$$

i.e., the $i^{\rm{th}}$ diagonal entry of the reduced electronic density matrix. State populations obtained over the propagation time can be straightforwardly fitted to exponential functions or applied within more sophisticated kinetic models to extract transition rates and excited-state lifetimes.


## Quantum resource estimation
The remainder of this notebook contains resource estimation analysis for the simulation of vibronic Hamiltonians using the algorithm described in [arXiv:2403.10504](https://arxiv.org/abs/2411.13669).

The algorithm utilizes a real-space discretization of the vibrational degrees of freedom, where each mode is resolved on a grid of $K = 2^k$ points each, requiring $k$ qubits. The $N$ electronic states are simply mapped to the computational basis states on a $log(N)$-qubit register, and hence $H$ acts on the space of $M \log(K) + \log(N)$ qubits within this representation.

The systems used for resources estimation for simulting vibronic dynamics are:

<div style="display: flex; justify-content: space-between; align-items: flex-start; gap: 2em;">

  <figure style="text-align: center; margin: 0;">
    <img src="diagrams/anth_monomer.png" style="height: 450px; width: auto;"/>
    <figcaption style="font-size: 0.9em; margin-top: 0.5em;">
      <b>Fig. 1:</b> Intramolecular singlet fission in (NO)4-Anth, an  anthracene modified with four N-oxyl moieties.
    </figcaption>
  </figure>

  <figure style="text-align: center; margin: 0;">
    <img src="diagrams/anth_dimer.png" style="height: 450px; width: auto;"/>
    <figcaption style="font-size: 0.9em; margin-top: 0.5em;">
      <b>Fig. 2:</b> Triplet exciton separation in a (NO)4-Anth dimer system.
    </figcaption>
  </figure>

  <figure style="text-align: center; margin: 0;">
    <img src="diagrams/anth_c60.png" style="height: 450px; width: auto;"/>
    <figcaption style="font-size: 0.9em; margin-top: 0.5em;">
      <b>Fig. 3:</b> Charge transfer between anthracene and fullerene C(60), emulating a chromophore-acceptor interface, referred to as Anth/C(60).
    </figcaption>
  </figure>
  
</div>

Below, we summarize the vibronic Hamiltonian specifications for these systems.


| **System**                                | **Process of interest**      | **Number of states** | **Number of modes**|                    
|-------------------------------------------|------------------------------|----------------------|--------------------|
| (NO)₄-Anth                                |   Singlet fission            | 5                    | 19                 |
| (NO)₄-Anth Dimer                          |   Triplet separation         | 6                    | 21                 |
| Anth/$C_{60}$ (reduced dimensionality)    |   Charge separation          | 4                    | 11                 |
| Anth/$C_{60}$ (full dimensionality)       |   Charge separation          | 4                    | 246                |


In [72]:
''' 
Uncomment the molecule that resource estimation should be performed for.
'''

mol = 'n4o4a_sf' 
# mol = 'n4o4a_ts'            
# mol = 'anthra-c60_ct_M=11'
# mol = 'anthra-c60_ct_M=246'

k = 4 # Number of Qubits for discretization of each mode

filehandler = open(f'model_params/{mol}.pkl', 'rb')
omegas, couplings = pickle.load(filehandler)
filehandler.close()

QVC = bool(2 in couplings)
# Check if the model has contain Quadratic Couplings
omegas = np.array(omegas)
lambdas = couplings[0]
alphas = couplings[1]

N, M = alphas.shape[0], alphas.shape[2]
log_N = math.ceil(math.log2(N))

if QVC:
    betas = couplings[2]
else:
    betas = np.zeros((N, N, M, M))

print(f'Loaded {mol} Hamiltonian with {M} modes and {N} states.')

Loaded n4o4a_sf Hamiltonian with 19 modes and 5 states.


/var/folders/zv/cfr06cz541b53yzvds885cb00000gq/T/ipykernel_39002/4233113550.py:13: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  omegas, couplings = pickle.load(filehandler)


# Fragmentation

<div style="display: flex; justify-content: space-between; align-items: flex-start;">

<div style="width: 55%; padding-right: 2em;">

Fragmentation is used to break the full vibronic Hamiltonian into smaller, structured components that are easier to simulate on a quantum computer. For the vibronic Hamiltonian, fragments are found such that each of them is block diagonal in the electronic basis.

This structure enables efficient implementation of the time evolution operator for each fragment. For fragmenting the vibronic hamiltonian, the first $L$ fragments are for decomposition of $V$ and the last fragment $H_T = T$:

$$
H = \sum_{m=1}^{L} H_m + H_T ,
$$

The `vibronic_fragments` utility in **Pennylane** implements the decomposition of the Hamiltonian into $L = N + 1$ fragments, represented by `RealspaceMatrix` objects, using the **Original Grouping (OG)** method described in the equations 7 - 9 of the paper.  


</div>

<figure style="width: 40%; margin: 0;">
  <img src="diagrams/fragmentation.png" style="width: 70%;"/>
  <figcaption style="font-size: 0.9em; margin-top: 0.5em; width: 70%; text-align: center;">
    <b>Fig. 4:</b> Visualization of the Hamiltonian fragmentation process for a 4-state system. Each color for the blocks corresponds to a distinct fragment. The fragmentation preserves block-diagonality and allows each fragment to be simultaneously diagonalized using just Clifford operations. 

  </figcaption>
</figure>

</div>

In [73]:
fragments = vibronic_fragments(N, M, freqs=omegas, taylor_coeffs=[lambdas, alphas, betas])
print(f'Decomposed Hamiltonian into {len(fragments)} fragments.')

Decomposed Hamiltonian into 9 fragments.


# Single Trotter Step

We use the **second-order Trotter product formula** to implement the propagator for the the full vibronic Hamiltonian \( H = T + V \):

$$
U_2(\Delta t) = e^{-i H \Delta t} + \mathcal{O}(\Delta t^3) \approx \left( \prod_{m=1}^{L} e^{-i H_m \Delta t / 2} \right) \cdot e^{-i H_T \Delta t} \cdot \left( \prod_{m=1}^{L} e^{-i H_m \Delta t / 2} \right),
$$

where:
- $\{H_m\}_{m=1}^L$ are the **potential energy fragments**,
- $H_T$ is the **kinetic energy fragment**,
- $\Delta t$ is the Trotter step size.

Hence, we can implement a single trotter step by implementing propagators of the potential energy and the kinetic energy fragments.

### Cost Metric

We take the Toffoli gate count, alongside qubit count, as our primary cost metric because Toffolis serve as a higher‐level proxy for non‐Clifford resources in many quantum algorithms. In a fault‐tolerant setting, each Toffoli (CCNOT) must be synthesized from Clifford gates and T‐states, and since T‐states themselves require costly magic‐state distillation, a single Toffoli often corresponds to multiple T gates plus ancilla overhead. Moreover, Toffoli‐based constructions are used for the arithmetic and modular‐arithmetic subroutines (e.g., adders, squaring, multiplier circuits) in quantum chemistry and optimization routines, thus, minimizing Toffoli count directly reduces the distillation workload and overall circuit depth. 

In [74]:
my_gs = {'X', 'Z', 'Y', 'S', 'Hadamard', 'CNOT', 'T', 'Toffoli'}

def print_Qubit_Toff(resources):
    '''Function for printing the resources for a circuit'''
    qubit_count = resources.qubit_manager.total_qubits
    toffoli_count = resources.clean_gate_counts.get("Toffoli", 0)
    
    if toffoli_count > 9999:
        toffoli_count = f"{toffoli_count:.3E}"
    
    print(f'Qubit Count = {qubit_count}')
    print(f'Toffoli Count = {toffoli_count}')

## Potential Part
Implementing each potential fragment $ e^{iH_m} = \sum_j \ket{j}\bra{j} \otimes e^{i V_{jj}^m} $ is equal to implementing

$$
\prod_{|\vec{\alpha}| \leq d} \sum_{j,\boldsymbol{x}} \ket{j}\bra{j} \otimes \ket{\boldsymbol{x}}\bra{\boldsymbol{x}} \cdot \exp\left(i \Delta^{|\vec{\boldsymbol{\alpha}}|} \cdot c_{\vec{\boldsymbol{\alpha}}}^{(j,j)}\, \boldsymbol{x}^{\vec{\boldsymbol{\alpha}}}\right)
$$ 


Each term $\sum_{j,\boldsymbol{x}} \ket{j}\bra{j} \otimes \ket{\boldsymbol{x}}\bra{\boldsymbol{x}} \cdot \exp\left(i \Delta^{|\vec{\boldsymbol{\alpha}}|} \cdot c_{\vec{\boldsymbol{\alpha}}}^{(j,j)}\, \boldsymbol{x}^{\vec{\boldsymbol{\alpha}}}\right)$ can be implemented using the following steps:

1. Load the binary representation of the coefficients $c_{\vec{\boldsymbol{\alpha}}}^{(j,j)}$ into an ancillary register controlled on the corresponding electronic state using QROM. $\Delta^{|\vec{\boldsymbol{\alpha}}|}$ is absorbed into the coefficients.

2. Compute the corresponding variable $\boldsymbol{x}^{\vec{\boldsymbol{\alpha}}}$ in another register using quantum arithmetic by taking the product between the corresponding mode registers.

3. Compute the product between the coefficient and the variable: $c_{\vec{\boldsymbol{\alpha}}}^{(j,j)}\, \boldsymbol{x}^{\vec{\boldsymbol{\alpha}}}$ and simultaneously perform a addition to the resource state to get a phase of $c_{\vec{\boldsymbol{\alpha}}}^{(j,j)}\, \boldsymbol{x}^{\vec{\boldsymbol{\alpha}}}$.

4. Uncompute all intermediate results.



### For Model systems
In many practical vibronic Hamiltonians, including the systems used for for resource estimation here, each potential fragment  $H_m$ contains non trivial coefficients only for linear and diagonal quadratic position operators. That is, the vibrational part of each fragment is of the form:

$$
V_{jj}^m = \sum_r c_r Q_r + \sum_r c_r Q_r^2,
$$
Each exponential term of the form $\exp(i \Delta^{|\vec{\alpha}|} \cdot c_r x_r^{|\vec{\alpha}|})$ is implemented by:

1. Loading the coefficient $c_r$ into an ancillary register using QROM, controlled by the electronic state. $\Delta^{|\vec{\alpha}|}$ is absorbed into the coefficients. 
2. Computing the $x_r^2$ (for $Q_r^2$ terms) using an Out-of-Place Squaring gate from the mode register to the scratch register.
3. Multiplying $x_r$ or $x_r^2$ with $c_r$ and performing an in-place addition to the resource register to accumulate the phase.
4. Uncomputing all intermediate results to preserve reversibility.

This structure is reflected in the circuit diagrams:
- **Figure 5** shows the potential fragment implementation algorithm.
- **Figure 6** details the construction of the multiply-add subroutine (Mult Add gate) using controlled in-place additions to the resource register.

<div style="display: flex; justify-content: space-between; align-items: flex-start; gap: 2em;">

  <figure style="text-align: center; width: 1000px; margin: 0;">
    <img src="diagrams/circuit1.png" width="100%"/>
    <figcaption style="font-size: 0.9em; margin-top: 0.5em;">
      <b>Fig. 5:</b> Quantum circuit for implementing potential energy fragments.
    </figcaption>
  </figure>

  <figure style="text-align: center; width: 600px; margin: 0;">
    <img src="diagrams/circuit2.png" width="100%"/>
    <figcaption style="font-size: 0.9em; margin-top: 0.5em;">
      <b>Fig. 6:</b> Mult Add Gate Construction.
    </figcaption>
  </figure>

</div>

### Ancilla state prep
`ResourcePhaseGradient` prepares the resource state used for the implementing the phase gradient:

$$
|R\rangle = \sum_{j=0}^{2^b-1} e^{2\pi i j / 2^b} |j\rangle.
$$

We use `b = 20` for a precision of $10^{-6}$ in the phase added using the phase gradient in the algorithm.

In [76]:
b = 20
phase_grad_wires = [f"pg_{i}" for i in range(b)]

def initial_circ():
    re.ResourcePhaseGradient(num_wires=b, wires=phase_grad_wires)

# We change the default single qubit rotation precision to 1e-15 so that the error
# from the roations can be neglected for error in the phase added.
epsilon = 1e-15

res_initial = re.estimate_resources(
    initial_circ, 
    gate_set=my_gs, 
    single_qubit_rotation_error=epsilon,
)()

print(res_initial)

AttributeError: module 'pennylane.labs.resource_estimation' has no attribute 'estimate_resources'

Estimating the cost of implementing linear $Q_r$ terms:

In [75]:
clean_factor = False

electronic_wires = [f"e_{i}" for i in range(log_N)]
coeff_wires = [f"c_{i}" for i in range(b)]
total_mode_wires = [f"m_{i}" for i in range(k*M)]
mode_wires = total_mode_wires[:k]
scratch_wires = [f"s_{i}" for i in range(2*k)] 

def Q_cir():
    re.ResourceQROM(num_bitstrings=log_N, size_bitstring=b, clean=clean_factor, wires=electronic_wires + coeff_wires)

    for i in range(k):
        ctrl_wire = [mode_wires[i]]
        target_wires = coeff_wires[:b-i] + phase_grad_wires[:b-i]
        re.ResourceControlled(re.ResourceSemiAdder(max_register_size=b - i), num_ctrl_wires=1, num_ctrl_values=0, wires=target_wires+ctrl_wire)

    re.ResourceAdjoint(re.ResourceQROM(num_bitstrings=log_N, size_bitstring=b, clean=clean_factor, wires=electronic_wires + coeff_wires))
     
    re.ResourceIdentity(wires = scratch_wires + total_mode_wires[k:])
    

res_Q = re.estimate_resources(Q_cir, gate_set=my_gs)()
print_Qubit_Toff(res_Q)

AttributeError: module 'pennylane.labs.resource_estimation' has no attribute 'estimate_resources'

Estimate the cost of implementing quadratic $Q_r^2$ terms:

In [ ]:
def Qsquared_cir():
    re.ResourceQROM(num_bitstrings=log_N, size_bitstring=b, clean=clean_factor, wires=electronic_wires + coeff_wires)

    re.ResourceOutOfPlaceSquare(register_size=k, wires=scratch_wires+mode_wires)

    for i in range(2*k):
        ctrl_wire = [f"s_{i}"]
        target_wires = coeff_wires[:b-i] + phase_grad_wires[:b-i]
        re.ResourceControlled(re.ResourceSemiAdder(max_register_size=b - i), num_ctrl_wires=1, num_ctrl_values=0, wires=target_wires + ctrl_wire)

    re.ResourceAdjoint(re.ResourceOutOfPlaceSquare(register_size=k, wires=scratch_wires+mode_wires))

    re.ResourceAdjoint(re.ResourceQROM(num_bitstrings=log_N, size_bitstring=b, clean=clean_factor, wires=electronic_wires + coeff_wires))

    re.ResourceIdentity(wires = total_mode_wires[k:])

res_Qsquared = re.estimate_resources(Qsquared_cir, gate_set=my_gs)()
print_Qubit_Toff(res_Qsquared)

Qubit Count = 1053
Toffoli Count = 546


The `count_nonzero_Q_terms` function counts the number of $Q_r$ and  $Q_r^2$ terms over all fragments. These counts determine how many times the circuits `Q_cir` and `Qsquared_cir` will be required for implementing all the fragments of the potential part.


In [ ]:
def count_nonzero_Q_terms(matrix, threshold=1e-8):
    Q_modes = set()
    QQ_modes = set()

    for rs_sum in matrix._blocks.values():
        for op in rs_sum.ops:
            if op.ops == ("Q",):
                for index, val in op.coeffs.nonzero(threshold).items():
                    Q_modes.add(index[0])
            elif op.ops == ("Q", "Q"):
                for index, val in op.coeffs.nonzero(threshold).items():
                    QQ_modes.add(tuple(index))

    return  Q_modes, QQ_modes

num_Q = 0
num_QQ = 0
for i in range(len(fragments)):
    l,q = count_nonzero_Q_terms(fragments[i])
    print(f'Fragment {i+1} has {len(l)} Q_i terms and {len(q)} Q_i^2 terms.')
    num_Q += len(l)
    num_QQ += len(q)
print(f'Total number of Q terms: {num_Q}, Total number of Q^2 terms: {num_QQ}')

Fragment 1 has 246 Q_i terms and 246 Q_i^2 terms.
Fragment 2 has 0 Q_i terms and 0 Q_i^2 terms.
Fragment 3 has 0 Q_i terms and 0 Q_i^2 terms.
Fragment 4 has 0 Q_i terms and 0 Q_i^2 terms.
Fragment 5 has 0 Q_i terms and 0 Q_i^2 terms.
Total number of Q terms: 246, Total number of Q^2 terms: 246


So multiplying the number of $Q_r$ and  $Q_r^2$ terms over all fragments by the cost of the circuits `Q_cir` and `Qsquared_cir` to get the T gate count of the potential part.

In [ ]:
res_Potential = num_Q *res_Q + num_QQ *res_Qsquared
print_Qubit_Toff(res_Potential)

Qubit Count = 1053
Toffoli Count = 207132


## Kinetic Part

We can use QFTs(Quantum Fourier Transforms) to diagonalize the momentum operator. In the discretized basis:

$$
P = \text{QFT}^\dagger \cdot X_{k-1} \cdot Q \cdot X_{k-1} \cdot \text{QFT},
$$

where $X_{k-1}$ is an  $X$ gate on the most significant qubit. Each $P_r^2$ term in the fourier basis can be simulated as a  $Q_r^2$ term in the position basis.

In [ ]:
'''
We use a different implementation of the QFT algorithm which is more resource efficient than
the default textbook implemention used in Pennylane Resource Estimation,
This comes from <https://arxiv.org/abs/1803.04933v2>.
'''

def AQFT_resource_decomp(num_wires, **kwargs):
    ceil_log_n = math.ceil(math.log2(num_wires))
    aux_qubit_count = num_wires + 3*ceil_log_n - 4  # (Section IV. )
    
    toff = re.ResourceToffoli.resource_rep()
    toff_count = 2 * num_wires*(ceil_log_n - 1)

    gate_list = [
        re.AllocWires(aux_qubit_count),
        re.GateCount(toff, toff_count),
        re.FreeWires(aux_qubit_count),
    ]
    return gate_list

re.ResourceQFT.set_resources(AQFT_resource_decomp)
# re.ResourceQFT.set_resources(re.ResourceQFT.default_resource_decomp)  # reset to default QFT cost

In [ ]:
def kinetic_cir():
    for i in range(M):
        re.ResourceQFT(num_wires=k, wires=total_mode_wires[i*k: k*(i + 1)])

    for i in range(M):
        mode_wires = total_mode_wires[i*k: k*(i + 1)]
        re.ResourceOutOfPlaceSquare(register_size=k, wires=mode_wires+scratch_wires)

        for j in range(2*k):
            ctrl_wire = [f"s_{j}"]
            target_wires = coeff_wires[:b-j] + phase_grad_wires[:b-j]
            re.ResourceControlled(re.ResourceSemiAdder(max_register_size=b - j), num_ctrl_wires=1, num_ctrl_values=0, wires=target_wires + ctrl_wire)

        re.ResourceAdjoint(re.ResourceOutOfPlaceSquare(register_size=k, wires=mode_wires+scratch_wires))

    for i in range(M):
        re.ResourceAdjoint(re.ResourceQFT(num_wires=k), wires=total_mode_wires[i*k: k*(i + 1)])

    re.ResourceIdentity(wires=electronic_wires)

res_kinetic = re.estimate_resources(kinetic_cir, gate_set=my_gs)()
print_Qubit_Toff(res_kinetic)

Qubit Count = 1053
Toffoli Count = 138252


## trotter step total cost

For each trotter step, all potential energy fragments $H_m$ are applied **twice**, each for **half the time**, and the kinetic energy fragment $H_T$ is applied once.

In [ ]:
res_step = res_kinetic + 2*res_Potential
print_Qubit_Toff(res_step)

Qubit Count = 1053
Toffoli Count = 552516


### New Fragmentation Schemes

In Trotterized quantum simulation, the way we fragment the Hamiltonian directly affects both the simulation cost and the accuracy of the evolution. The Original Grouping method may not minimize gate counts for simulation of every system of interest, so we present an alternative method of fragmentation. In particular, **each mode may be implemented multiple times** in the OG method, leading to high repetition of resource-intensive subroutines.

To address this, A system with $p$ vibrational modes can be fragmented such that each vibrational mode contributes two types of fragments: one involving $Q_r$ and another involving $Q_r^2$. The first $M$ fragments correspond to the linear terms:

$$
H^{(r)} = \sum_{i,j} h_{ij}^{(r)}\, \ket{i}\bra{j} \otimes Q_r, \qquad r = 1,\ldots,M,
$$

and the next $M$ fragments correspond to the quadratic terms:

$$
H^{(M + r)} = \sum_{i,j} h_{ij}^{(M +r)}\, \ket{i}\bra{j} \otimes Q_r^2, \qquad r = 1,\ldots,M.
$$

Since each $Q_r$ and $Q_r^2$ appears exactly once in the fragments, this guarantees that the linear and quadratic circuits per Trotter step, as discussed in the next section, are optimal.

Next, we combine multiple such fragments into a single fragment to reduce the number of terms without increasing the number of linear and quadratic circuits per Trotter step.
If $H_e^{(r)}$ and $H_e^{(s)}$ commute, they can be simultaneously diagonalized by a unitary transformation $U$. Consequently, the combined fragment can be block-diagonalized in the electronic subspace by the same unitary $U$. Hence, A sufficient condition for two fragments $H^{(r)}$ and $H^{(s)}$ to be combined is that their electronic parts commute.

In [ ]:
res_Potential_Mode_Based = M *res_Q + M *res_Qsquared
print_Qubit_Toff(res_Potential_Mode_Based)

Qubit Count = 1053
Toffoli Count = 207132


In [ ]:
res_step_Mode_Based = res_kinetic + 2*res_Potential_Mode_Based
print_Qubit_Toff(res_step_Mode_Based)

Qubit Count = 1053
Toffoli Count = 552516


# Number of Trotter steps

The algorithm uses the **second-order Trotter–Suzuki formula** to approximate time evolution under the full Hamiltonian $H = \sum_{m=1}^{L} H_m$. This corresponds to evolution under an **effective Hamiltonian**:

$$
U(\Delta t) = e^{i H_{\text{eff}} \Delta t}, \quad \text{where} \quad H_{\text{eff}} = H + \hat{\epsilon}.
$$

The leading-order error operator $\hat{\epsilon}$ for the second-order formula scales as:

$$
\hat{\epsilon} = -\frac{\Delta t^2}{24} \sum_{i=1}^{L-1} \sum_{j=i+1}^{L} \left[H_i + 2 \sum_{k=j+1}^L H_k, [H_i, H_j] \right] + \mathcal{O}(\Delta t^4).
$$

This implies that the **local (per-step) error** in operator norm is:

$$
\| U_2(\Delta t) - e^{iH \Delta t} \| = \mathcal{O}(\Delta t^3).
$$

#### Global Error Over Total Time $T$

Suppose we simulate for total time $T$, using $N$ steps of size $\Delta t = T / N$. Then the **global error** accumulates as:

$$
\| U_2(\Delta t)^N - e^{iHT} \| = \mathcal{O}(T \Delta t^2).
$$

That is, the global error $\varepsilon$ scales quadratically in $\Delta t$:

$$
\varepsilon \sim C \cdot T \cdot \Delta t^2,
$$

for some constant $C$ determined by nested commutators of the fragments.

In the Pennylane resource estimation framework, the effective Hamiltonian $H_{\text{eff}}$ is constructed explicitly from building the the error operator using the `trotter_error.trotter_error` module in Pennylane and adding it to the sum of `vibronic_fragments`.

In [ ]:
sim_deltaT = 0.1
sim_time = 100

# err = trotter_error(fragments, sim_deltaT)
# h_op = sum(fragments, RealspaceMatrix.zero(states=2**log_N, modes=M))
# h_eff = h_op + err

<div style="display: flex; justify-content: space-between; align-items: flex-start;">

<div style="width: 40%; padding-right: 2em;">

To quantify the Trotter error, we first simulate the dynamics under both the **exact Hamiltonian** $H$ and the **effective Hamiltonian** $H_{\text{eff}}$ for **reduced-mode systems** (e.g., 1–6 modes) using classical Methods like `MCTDH`. The error at time $t$ is given by the maximum difference in state population between the two evolutions.

Once we have the Trotter errors for small subsystems, we can extrapolate to larger systems to estimate the number of steps required to reach a target error $\varepsilon$

To apply this in practice, we simulate systems with increasing number of modes (e.g., 1 to 6) for a chosen `deltaT`(e.g. $\Delta t =0.1$), calculate the maximum error across all states and over total time, and **fit an extrapolation curve** for error vs. number of modes. 
This allows us to estimate the error we would expect in the **full model** (e.g., 11-mode Anthra-$C_{60}$ system).

</div>

<figure style="width: 40%; margin: 0;">
  <img src="diagrams/extrapolation.png" style="width: 70%;"/>
  <figcaption style="font-size: 0.9em; margin-top: 0.5em; width: 70%; text-align: center;">
    <b>Fig. 8:</b> Plot showing the extrapolated error curve for the Anthra/C_60 system based on mode counts 1–6, including a best-fit line used to project the error at 11 modes. 

  </figcaption>
</figure>

</div>

In [ ]:
def extrapolated_error(mol, fragmentation, sim_time):
    errors = {
        ("n4o4a_sf",            "OG",         100): 2.622e-3,
        ("n4o4a_sf",            "Mode-Based", 100): 2.849e-3,
        ("n4o4a_ts",            "OG",         100): 1.231e-4,
        ("n4o4a_ts",            "Mode-Based", 100): 1.935e-4,
        ("anthra-c60_ct_M=11",  "OG",         100): 3.02e-5,
        ("anthra-c60_ct_M=11",  "Mode-Based", 100): 3.02e-5,
        ("anthra-c60_ct_M=246", "OG",         100): 3.020e-5,
        ("anthra-c60_ct_M=246", "Mode-Based", 100): 3.020e-5,
        ("n4o4a_ts",            "OG",         500): 8.113e-4,
        ("n4o4a_ts",            "Mode-Based", 500): 7.395e-4,
    }

    error = errors.get((mol, fragmentation, sim_time))
    print(
        f"Trotter error for a {sim_time} fs simulation with deltaT = {sim_deltaT}, "
        f"using {fragmentation} fragmentation is {error}."
    )
    return error

fragmentation = 'OG'
# fragmentation = 'Mode-Based'

sim_error = extrapolated_error(mol, fragmentation, sim_time)

Trotter error for a 100 fs simulation with deltaT = 0.1, using OG fragmentation is 0.002622.


### Solving for $\Delta t$ Given an Error Tolerance

Suppose we know that using a step size $\Delta t_1$ gives global error $\varepsilon_1$ for a simulation Time T. We want to choose a new step size $\Delta t_2$ such that the global error is reduced to $\varepsilon_2$ for the same simulation time T.

Using the quadratic scaling:

$$
\varepsilon \propto \Delta t^2 \quad \Rightarrow \quad \frac{\varepsilon_2}{\varepsilon_1} = \left( \frac{\Delta t_2}{\Delta t_1} \right)^2
$$

Solving for $\Delta t_2$ gives:

$$
\Delta t_2 = \Delta t_1 \cdot \sqrt{ \frac{\varepsilon_2}{\varepsilon_1} }.
$$

Once $\Delta t_2$ is known, the required number of steps is:

$$
N_2 = \frac{T}{\Delta t_2} = \frac{T}{\Delta t_1} \cdot \sqrt{ \frac{\varepsilon_1}{\varepsilon_2}}.
$$

In [ ]:
def num_steps(req_error, sim_error, sim_deltaT, sim_time):
    return math.ceil(sim_time / sim_deltaT * (sim_error / req_error)**0.5)

req_error = 0.01 # 1 percent error
nsteps = num_steps(req_error, sim_error, sim_deltaT, sim_time)

print(f'{nsteps} Trotter steps are required for a {100*req_error}% error tolerance for simulation of the hamiltonian decomposed by the {fragmentation} fragmentation scheme for {sim_time}fs.')

55 Trotter steps are required for a 1.0% error tolerance for simulation of the hamiltonian decomposed by the Mode-Based fragmentation scheme for 100fs.


In [ ]:
if fragmentation == 'OG':
    res_total = res_initial + nsteps * res_step
if fragmentation == 'Mode-Based':
    res_total = res_initial + nsteps * res_step_Mode_Based

print_Qubit_Toff(res_total)

Qubit Count = 1053
Toffoli Count = 30388380


# Results

The estimated implementation costs of our quantum algorithm for the systems of interest are:

| **System**                              | **N** | **M** | **Time (fs)** | **Error** | **Qubit Count** | **Step Toffoli Count** (OG) | **Step Toffoli Count** (Mode Based) | **Number of Steps** (OG) | **Number of Steps** (Mode Based) | **Total Toffoli Count** (OG) | **Total Toffoli Count** (Mode Based) |
|----------------------------------------|-------|-------|------------|--------------------------|------------|------------------------|--------------------------|----------------------|------------------------|----------------------|------------------------|
| (NO)₄-Anth                             | 5     | 19    | 100        | 10%                       | 146        | $3.37 \times 10^{4}$   | $8.98 \times 10^{4}$     |        162       |       169          | $5.47 \times 10^{6}$ |  $3.54 \times 10^{6}$ |
| (NO)₄-Anth                             | 5     | 19    | 100        | 1%                       | 146        | $3.37 \times 10^{4}$   | $2.09 \times 10^{4}$     | 513                  | 534                    | $1.73 \times 10^{7}$ | $1.11 \times 10^{7}$   |
| (NO)₄-Anth Dimer                       | 6     | 21    | 100        | 1%                       | 154        | $2.48 \times 10^{4}$   | $2.31 \times 10^{5}$     | 111                  | 140                    | $2.76 \times 10^{6}$ | $3.24 \times 10^{6}$   |
| (NO)₄-Anth Dimer                       | 6     | 21    | 500        | 1%                       | 154        | $1.07 \times 10^{5}$   | $1.07 \times 10^{5}$     |  1425                | 1360           | $3.54 \times 10^{7}$         | $3.15 \times 10^{7}$ | 
| Anth/$C_{60}$ (reduced dimensionality) | 4     | 11    | 100        | 1%                       | 113        | $1.20 \times 10^{4}$   | $1.20 \times 10^{4}$     | 55                   | 55                     | $6.62 \times 10^{5}$ | $6.62 \times 10^{5}$   |
| Anth/$C_{60}$ (full dimensionality)    | 4     | 246   | 100        | 1%                       | 1053       | $1.16 \times 10^{6}$   | $1.16 \times 10^{6}$     | 99                   | 99                     | $2.66 \times 10^{7}$ | $2.66 \times 10^{7}$   |
